# New Section

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns


import sys
import os
import gc

import numpy as np
import pandas as pd

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)



import sklearn.model_selection



from tqdm import tqdm
import random 
import datetime
import joblib



#import lightgbm as lgb


'''
#LOCAL
ROOT_DIR = os.path.expanduser('~')

folder = ROOT_DIR + '/Downloads/m5-forecasting-accuracy/'

calendar = pd.read_csv(folder+'calendar.csv')
sales_train_evaluation = pd.read_csv(folder+'sales_train_evaluation.csv')
sales_train_validation = pd.read_csv(folder+'sales_train_validation.csv')
sample_submission = pd.read_csv(folder+'sample_submission.csv')
sell_prices = pd.read_csv(folder+'sell_prices.csv')
'''


#'''
#SAGEMAKER:
#!conda install -c conda-forge lightgbm

import boto3
import sagemaker
from sagemaker import get_execution_role


# sagemaker session, role
sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()


# S3 bucket name
bucket = 'm5-forecasting-accuracy'
subfolder = ''
key='calendar'
conn = boto3.client('s3')
contents = conn.list_objects(Bucket=bucket, Prefix=subfolder)['Contents']

calendar = pd.read_csv('s3://{}/{}'.format(bucket, 'calendar.csv'))
sales_train_evaluation = pd.read_csv('s3://{}/{}'.format(bucket, 'sales_train_evaluation.csv'))
sales_train_validation = pd.read_csv('s3://{}/{}'.format(bucket, 'sales_train_validation.csv'))
sample_submission = pd.read_csv('s3://{}/{}'.format(bucket, 'sample_submission.csv'))
sell_prices = pd.read_csv('s3://{}/{}'.format(bucket, 'sell_prices.csv'))
#'''


In [2]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df


# function to read the data and merge it (ignoring some columns, this is a very fst model)


def read_data():
    INPUT_DIR = '../data/input/'
    print('Reading files...')
    calendar = pd.read_csv(f'{INPUT_DIR}calendar.csv')
    calendar = reduce_mem_usage(calendar)
    print('Calendar has {} rows and {} columns'.format(calendar.shape[0], calendar.shape[1]))
    sell_prices = pd.read_csv(f'{INPUT_DIR}sell_prices.csv')
    sell_prices = reduce_mem_usage(sell_prices)
    print('Sell prices has {} rows and {} columns'.format(sell_prices.shape[0], sell_prices.shape[1]))
    sales_train_validation = pd.read_csv(f'{INPUT_DIR}sales_train_validation.csv')
    print('Sales train validation has {} rows and {} columns'.format(sales_train_validation.shape[0], sales_train_validation.shape[1]))
    submission = pd.read_csv(f'{INPUT_DIR}sample_submission.csv')
    return calendar, sell_prices, sales_train_validation, submission


def melt_and_merge(calendar, sell_prices, sales_train_validation, submission, nrows = 55000000, merge = False):
    
    # melt sales data, get it ready for training
    sales_train_validation = pd.melt(sales_train_validation, id_vars = ['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], var_name = 'day', value_name = 'demand')
    print('Melted sales train validation has {} rows and {} columns'.format(sales_train_validation.shape[0], sales_train_validation.shape[1]))
    sales_train_validation = reduce_mem_usage(sales_train_validation)
    
    # seperate test dataframes
    test1_rows = [row for row in submission['id'] if 'validation' in row]
    test2_rows = [row for row in submission['id'] if 'evaluation' in row]
    test1 = submission[submission['id'].isin(test1_rows)]
    test2 = submission[submission['id'].isin(test2_rows)]
    
    # change column names
    test1.columns = ['id', 'd_1914', 'd_1915', 'd_1916', 'd_1917', 'd_1918', 'd_1919', 'd_1920', 'd_1921', 'd_1922', 'd_1923', 'd_1924', 'd_1925', 'd_1926', 'd_1927', 'd_1928', 'd_1929', 'd_1930', 'd_1931', 
                      'd_1932', 'd_1933', 'd_1934', 'd_1935', 'd_1936', 'd_1937', 'd_1938', 'd_1939', 'd_1940', 'd_1941']
    test2.columns = ['id', 'd_1942', 'd_1943', 'd_1944', 'd_1945', 'd_1946', 'd_1947', 'd_1948', 'd_1949', 'd_1950', 'd_1951', 'd_1952', 'd_1953', 'd_1954', 'd_1955', 'd_1956', 'd_1957', 'd_1958', 'd_1959', 
                      'd_1960', 'd_1961', 'd_1962', 'd_1963', 'd_1964', 'd_1965', 'd_1966', 'd_1967', 'd_1968', 'd_1969']
    
    # get product table
    product = sales_train_validation[['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id']].drop_duplicates()
    
    # merge with product table
    test2['id'] = test2['id'].str.replace('_evaluation','_validation')
    test1 = test1.merge(product, how = 'left', on = 'id')
    test2 = test2.merge(product, how = 'left', on = 'id')
    test2['id'] = test2['id'].str.replace('_validation','_evaluation')
    
    # 
    test1 = pd.melt(test1, id_vars = ['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], var_name = 'day', value_name = 'demand')
    test2 = pd.melt(test2, id_vars = ['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], var_name = 'day', value_name = 'demand')
    
    sales_train_validation['part'] = 'train'
    test1['part'] = 'test1'
    test2['part'] = 'test2'
    
    data = pd.concat([sales_train_validation, test1, test2], axis = 0)
    
    del sales_train_validation, test1, test2
    
    # get only a sample for fst training
    data = data.loc[nrows:]
    
    # drop some calendar features
    calendar.drop(['weekday', 'wday', 'month', 'year'], inplace = True, axis = 1)
    
    # delete test2 for now
    data = data[data['part'] != 'test2']
    
    if merge:
        # notebook crash with the entire dataset (maybee use tensorflow, dask, pyspark xD)
        data = pd.merge(data, calendar, how = 'left', left_on = ['day'], right_on = ['d'])
        data.drop(['d', 'day'], inplace = True, axis = 1)
        # get the sell price data (this feature should be very important)
        data = data.merge(sell_prices, on = ['store_id', 'item_id', 'wm_yr_wk'], how = 'left')
        print('Our final dataset to train has {} rows and {} columns'.format(data.shape[0], data.shape[1]))
    else: 
        pass
    
    gc.collect()
    
    return data
        


**Load the data**

In [3]:
submission = sample_submission

In [4]:
sales_train_validation

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,d_11,d_12,d_13,d_14,d_15,d_16,d_17,d_18,d_19,d_20,d_21,d_22,d_23,d_24,d_25,d_26,d_27,d_28,d_29,d_30,d_31,d_32,d_33,d_34,d_35,d_36,d_37,d_38,d_39,d_40,d_41,d_42,d_43,d_44,d_45,d_46,d_47,d_48,d_49,d_50,d_51,d_52,d_53,d_54,d_55,d_56,d_57,d_58,d_59,d_60,d_61,d_62,d_63,d_64,d_65,d_66,d_67,d_68,d_69,d_70,d_71,d_72,d_73,d_74,d_75,d_76,d_77,d_78,d_79,d_80,d_81,d_82,d_83,d_84,d_85,d_86,d_87,d_88,d_89,d_90,d_91,d_92,d_93,d_94,d_95,d_96,d_97,d_98,d_99,d_100,d_101,d_102,d_103,d_104,d_105,d_106,d_107,d_108,d_109,d_110,d_111,d_112,d_113,d_114,d_115,d_116,d_117,d_118,d_119,d_120,d_121,d_122,d_123,d_124,d_125,d_126,d_127,d_128,d_129,d_130,d_131,d_132,d_133,d_134,d_135,d_136,d_137,d_138,d_139,d_140,d_141,d_142,d_143,d_144,d_145,d_146,d_147,d_148,d_149,d_150,d_151,d_152,d_153,d_154,d_155,d_156,d_157,d_158,d_159,d_160,d_161,d_162,d_163,d_164,d_165,d_166,d_167,d_168,d_169,d_170,d_171,d_172,d_173,d_174,d_175,d_176,d_177,d_178,d_179,d_180,d_181,d_182,d_183,d_184,d_185,d_186,d_187,d_188,d_189,d_190,d_191,d_192,d_193,d_194,d_195,d_196,d_197,d_198,d_199,d_200,d_201,d_202,d_203,d_204,d_205,d_206,d_207,d_208,d_209,d_210,d_211,d_212,d_213,d_214,d_215,d_216,d_217,d_218,d_219,d_220,d_221,d_222,d_223,d_224,d_225,d_226,d_227,d_228,d_229,d_230,d_231,d_232,d_233,d_234,d_235,d_236,d_237,d_238,d_239,d_240,d_241,d_242,d_243,d_244,...,d_1664,d_1665,d_1666,d_1667,d_1668,d_1669,d_1670,d_1671,d_1672,d_1673,d_1674,d_1675,d_1676,d_1677,d_1678,d_1679,d_1680,d_1681,d_1682,d_1683,d_1684,d_1685,d_1686,d_1687,d_1688,d_1689,d_1690,d_1691,d_1692,d_1693,d_1694,d_1695,d_1696,d_1697,d_1698,d_1699,d_1700,d_1701,d_1702,d_1703,d_1704,d_1705,d_1706,d_1707,d_1708,d_1709,d_1710,d_1711,d_1712,d_1713,d_1714,d_1715,d_1716,d_1717,d_1718,d_1719,d_1720,d_1721,d_1722,d_1723,d_1724,d_1725,d_1726,d_1727,d_1728,d_1729,d_1730,d_1731,d_1732,d_1733,d_1734,d_1735,d_1736,d_1737,d_1738,d_1739,d_1740,d_1741,d_1742,d_1743,d_1744,d_1745,d_1746,d_1747,d_1748,d_1749,d_1750,d_1751,d_1752,d_1753,d_1754,d_1755,d_1756,d_1757,d_1758,d_1759,d_1760,d_1761,d_1762,d_1763,d_1764,d_1765,d_1766,d_1767,d_1768,d_1769,d_1770,d_1771,d_1772,d_1773,d_1774,d_1775,d_1776,d_1777,d_1778,d_1779,d_1780,d_1781,d_1782,d_1783,d_1784,d_1785,d_1786,d_1787,d_1788,d_1789,d_1790,d_1791,d_1792,d_1793,d_1794,d_1795,d_1796,d_1797,d_1798,d_1799,d_1800,d_1801,d_1802,d_1803,d_1804,d_1805,d_1806,d_1807,d_1808,d_1809,d_1810,d_1811,d_1812,d_1813,d_1814,d_1815,d_1816,d_1817,d_1818,d_1819,d_1820,d_1821,d_1822,d_1823,d_1824,d_1825,d_1826,d_1827,d_1828,d_1829,d_1830,d_1831,d_1832,d_1833,d_1834,d_1835,d_1836,d_1837,d_1838,d_1839,d_1840,d_1841,d_1842,d_1843,d_1844,d_1845,d_1846,d_1847,d_1848,d_1849,d_1850,d_1851,d_1852,d_1853,d_1854,d_1855,d_1856,d_1857,d_1858,d_1859,d_1860,d_1861,d_1862,d_1863,d_1864,d_1865,d_1866,d_1867,d_1868,d_1869,d_1870,d_1871,d_1872,d_1873,d_1874,d_1875,d_1876,d_1877,d_1878,d_1879,d_1880,d_1881,d_1882,d_1883,d_1884,d_1885,d_1886,d_1887,d_1888,d_1889,d_1890,d_1891,d_1892,d_1893,d_1894,d_1895,d_1896,d_1897,d_1898,d_1899,d_1900,d_1901,d_1902,d_1903,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,0,0,2,0,2,2,0,0,0,1,1,0,2,0,1,1,2,0,1,0,0,0,2,1,0,1,1,2,0,0,0,0,0,0,0,1,0,1,0,1,0,3,1,1,0,1,1,2,0,0,0,0,1,1,0,0,0,0,3,0,1,0,0,0,0,1,1,1,0,1,0,2,0,0,0,0,2,0,0,0,0,1,1,2,0,0,0,0,2,0,0,1,1,1,1,0,0,0,0,0,1,2,2,0,1,0,0,0,0,1,2,1,0,0,0,0,0,1,0,3,0,1,2,1,0,3,0,0,0,1,0,2,2,1,0,0,1,2,0,1

In [5]:
data = melt_and_merge(calendar, sell_prices, sales_train_validation, submission, nrows = 27500000, merge = True)

MemoryError: Unable to allocate 445. MiB for an array with shape (1913, 30490) and data type object

**data preprocessing**

In [4]:
#https://www.kaggle.com/anshuls235/time-series-forecasting-eda-fe-modelling/notebook#2.-Downcasting

def downcast(df):

    cols = df.dtypes.index.tolist()
    types = df.dtypes.values.tolist()

    for i,t in enumerate(types):

        if 'int' in str(t):
            if df[cols[i]].min() > np.iinfo(np.int8).min and df[cols[i]].max() < np.iinfo(np.int8).max:
                df[cols[i]] = df[cols[i]].astype(np.int8)
            elif df[cols[i]].min() > np.iinfo(np.int16).min and df[cols[i]].max() < np.iinfo(np.int16).max:
                df[cols[i]] = df[cols[i]].astype(np.int16)
            elif df[cols[i]].min() > np.iinfo(np.int32).min and df[cols[i]].max() < np.iinfo(np.int32).max:
                df[cols[i]] = df[cols[i]].astype(np.int32)
            else:
                df[cols[i]] = df[cols[i]].astype(np.int64)

        elif 'float' in str(t):
            if df[cols[i]].min() > np.finfo(np.float16).min and df[cols[i]].max() < np.finfo(np.float16).max:
                df[cols[i]] = df[cols[i]].astype(np.float16)
            elif df[cols[i]].min() > np.finfo(np.float32).min and df[cols[i]].max() < np.finfo(np.float32).max:
                df[cols[i]] = df[cols[i]].astype(np.float32)
            else:
                df[cols[i]] = df[cols[i]].astype(np.float64)

        elif t == np.object:
            if cols[i] == 'date':
                df[cols[i]] = pd.to_datetime(df[cols[i]], format='%Y-%m-%d')
            else:
                df[cols[i]] = df[cols[i]].astype('category')

    return df  

In [5]:
sales = downcast(sales)
cal = downcast(cal)
price = downcast(price)

In [6]:
sales

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,4,0,0,0,0,3,3,0,1
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,1,2,1,1,0,0,0,0,0
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,2,0,0,0,2,3,0,1
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,1,0,4,0,1,3,0,2,6
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,2,1,0,0,2,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,FOODS_3_823_WI_3_evaluation,FOODS_3_823,FOODS_3,FOODS,WI_3,WI,0,0,2,2,...,1,0,3,0,1,1,0,0,1,1
30486,FOODS_3_824_WI_3_evaluation,FOODS_3_824,FOODS_3,FOODS,WI_3,WI,0,0,0,0,...,0,0,0,0,0,0,1,0,1,0
30487,FOODS_3_825_WI_3_evaluation,FOODS_3_825,FOODS_3,FOODS,WI_3,WI,0,6,0,2,...,0,0,1,2,0,1,0,1,0,2
30488,FOODS_3_826_WI_3_evaluation,FOODS_3_826,FOODS_3,FOODS,WI_3,WI,0,0,0,0,...,1,1,1,4,6,0,1,1,1,0


In [13]:
%time
for day in range(1942,1970):
  sales['d_' + str(day)] = 0
  sales['d_' + str(day)] = sales['d_' + str(day)].astype(np.int16)

CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 10.5 µs


In [14]:
%time
df_melt = pd.melt(sales, id_vars=[a for a in sales.columns if a.find("id")!=-1],
                           value_vars=[a for a in sales.columns if a.find("d_")==0], var_name='d', value_name='sales')

CPU times: user 3 µs, sys: 1e+03 ns, total: 4 µs
Wall time: 7.39 µs


MemoryError: Unable to allocate 458. MiB for an array with shape (60034810,) and data type object

In [ ]:
%time

df_melt = pd.merge(df_melt, cal, on='d', how='left')


In [ ]:
%time

df_melt = pd.merge(df_melt, price, on=['store_id','item_id','wm_yr_wk'], how='left')

In [ ]:
%time

df_melt["sell_price"].fillna(df_melt.groupby("id")["sell_price"].transform("mean"), inplace=True)

In [ ]:
df_melt.drop(columns=["date","weekday"], inplace=True)
df_melt['d'] = df_melt['d'].apply(lambda a: a.split('_')[1]).astype(np.int16)

In [ ]:
df_melt

In [ ]:
joblib.dump(df_melt, 'df_melt_lib.pkl')
df_melt.to_pickle("df_melt_pkl.pkl")


In [ ]:
**features engineering**

In [ ]:
#label encoding features

columns = list(df_melt.columns)
d_types = list(df_melt.dtypes.values)

for a, d_t in enumerate(d_types):
  if d_t.name == 'category':
    df_melt[columns[a]] = df_melt[columns[a]].cat.codes
        

In [ ]:
#lag features

lags = [28, 35, 42, 49, 56, 63, 70, 77]
for lag in tqdm(lags):
  df_melt["lag_" + str(lag)] = df_melt.groupby("id")["sales"].shift(lag).astype(np.float16)

100%|██████████| 8/8 [00:26<00:00,  3.36s/it]


In [ ]:
#rolling median features

df_melt['lag_28'] = df_melt.groupby('id')['sales'].transform(lambda x: x.shift(28))

df_melt['rolling_median_7'] = df_melt.groupby('id')['lag_28'].transform(lambda x: x.rolling(7).median())
df_melt['rolling_median_28'] = df_melt.groupby('id')['lag_28'].transform(lambda x: x.rolling(28).median())
df_melt['rolling_median_50'] = df_melt.groupby('id')['lag_28'].transform(lambda x: x.rolling(50).median())

In [ ]:
df_melt = df_melt[df_melt['d']>1050]
df_melt.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d,sales,wm_yr_wk,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price,lag_28,lag_35,lag_42,lag_49,lag_56,lag_63,lag_70,lag_77,rolling_median_7,rolling_median_28,rolling_median_50
32014500,14370,1437,3,1,0,0,1051,0,11347,1,12,2013,-1,-1,-1,-1,0,0,1,8.257812,0.0,0.0,1.0,2.0,1.0,1.0,0.0,0.0,1.0,0.5,0.0
32014501,14380,1438,3,1,0,0,1051,2,11347,1,12,2013,-1,-1,-1,-1,0,0,1,3.970703,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
32014502,14390,1439,3,1,0,0,1051,0,11347,1,12,2013,-1,-1,-1,-1,0,0,1,2.970703,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
32014503,14400,1440,3,1,0,0,1051,1,11347,1,12,2013,-1,-1,-1,-1,0,0,1,4.640625,2.0,3.0,5.0,8.0,1.0,3.0,5.0,2.0,1.0,2.0,2.0
32014504,14410,1441,3,1,0,0,1051,3,11347,1,12,2013,-1,-1,-1,-1,0,0,1,3.080078,0.0,1.0,3.0,1.0,4.0,3.0,1.0,1.0,0.0,0.0,1.0


In [ ]:
df_melt.isnull().any()

id                   False
item_id              False
dept_id              False
cat_id               False
store_id             False
state_id             False
d                    False
sales                False
wm_yr_wk             False
wday                 False
month                False
year                 False
event_name_1         False
event_type_1         False
event_name_2         False
event_type_2         False
snap_CA              False
snap_TX              False
snap_WI              False
sell_price           False
lag_28               False
lag_35               False
lag_42               False
lag_49               False
lag_56               False
lag_63               False
lag_70               False
lag_77               False
rolling_median_7     False
rolling_median_28    False
rolling_median_50    False
dtype: bool

In [ ]:
df_melt.to_pickle("df_melt")

In [ ]:
df_melt=pd.read_pickle("df_melt")

In [4]:
df_melt = pd.read_pickle("/content/drive/My Drive/M5 Forcasting/df_melt")

In [5]:
df_melt.shape

(28020310, 31)

**train-test split**

In [7]:
X_train = df_melt[df_melt["d"] < 1914].drop("sales", axis=1)
X_val =  df_melt[(df_melt['d']>=1914) & (df_melt['d']<=1941)].drop('sales',axis=1)
X_test = df_melt[df_melt["d"] >= 1942].drop("sales", axis=1)

y_train = df_melt[df_melt["d"] < 1914]["sales"]
y_val = df_melt[(df_melt['d']>=1914) & (df_melt['d']<=1941)]["sales"]

In [8]:
X_train.shape

(26312870, 30)

In [9]:
X_val.shape

(853720, 30)

compute the weights for level 12 series

In [10]:
price["id"] = price["item_id"] + "_" + price["store_id"] + "_evaluation"
cal["d"] = cal["d"].apply(lambda a: int(a.split("_")[1]))

In [11]:
#https://www.kaggle.com/qcw171717/other-naive-forecasts-submission-score/notebook

for day in tqdm(range(1886, 1914)):  
    wk_id = list(cal[cal["d"]==day]["wm_yr_wk"])[0]
    wk_price = price[price["wm_yr_wk"]==wk_id]
    df_sales = sales.merge(wk_price[["sell_price", "id"]], on=["id"], how='inner')
    df_sales["unit_sales_" + str(day)] = df_sales["sell_price"] * df_sales["d_" + str(day)]
    df_sales.drop(columns=["sell_price"], inplace=True)

100%|██████████| 28/28 [00:09<00:00,  2.98it/s]


In [12]:
col = [a for a in df_sales.columns if a.find("unit_sales")==0]
df_sales["sales"] = df_sales[col]
df_sales["weight"] = df_sales["sales"] / df_sales["sales"].sum()
df_sales.drop(columns=["sales", col[0]], axis=1, inplace=True)
df_sales["weight"] /= 12

In [13]:
aggregation_level = {2: ["state_id"], 3: ["store_id"], 4: ["cat_id"], 5: ["dept_id"], 
              6: ["state_id", "cat_id"], 7: ["state_id", "dept_id"], 8: ["store_id", "cat_id"], 9: ["store_id", "dept_id"],
              10: ["item_id"], 11: ["item_id", "state_id"]}

In [14]:
#function to calculate rmsse 

h = 28
n = 1913

def RMSSE(ground_truth, forecast, train_series):
    
    num = ((ground_truth - forecast)**2).sum(axis=1)
    den = 1/(n-1) * ((train_series[:, 1:] - train_series[:, :-1]) ** 2).sum(axis=1)
    rmsse = (1/h * num/den) ** 0.5

    return rmsse

In [15]:
# function to compute custom hyperparametrer tuning

def hyperparameter_tuning(X_train, y_train, model):

    model.fit(X_train, y_train)

    for d in range(1914, 1942):
        df_sales['F_' + str(d)] = model.predict(X_val[X_val['d']==d])
    
    data = df_sales[[a for a in df_sales.columns if a.find("d_") == 0 or a.find("F_") == 0]]
    data = data.sum()

    aggregated_df = pd.DataFrame(data).transpose()    
    aggregated_df["level"] = 1
    aggregated_df["weight"] = 1/12    
    columns = aggregated_df.columns  

    for lev in aggregation_level:
        df_t = df_sales.groupby(by=aggregation_level[lev]).sum().reset_index()
        df_t["level"] = lev
        aggregated_df = aggregated_df.append(df_t[columns])     

    train_columns = [a for a in df_sales.columns if a.find("d_") == 0 and int(a.split('_')[1]) < 1914]
    actual_value_columns = [a for a in df_sales.columns if a.find("d_") == 0 and int(a.split('_')[1]) in range(1914, 1942)]
    forecast_value_columns = [a for a in df_sales.columns if a.find("F_") == 0]    

    ground_truth_df = np.array(df_sales[actual_value_columns])
    forecast_df = np.array(df_sales[forecast_value_columns])
    train_series_df = np.array(df_sales[train_columns])

    ground_truth_agg_df = np.array(aggregated_df[actual_value_columns])
    forecast_agg_df = np.array(aggregated_df[forecast_value_columns])
    train_series_agg_df = np.array(aggregated_df[train_columns])

    df_sales["rmsse"] = RMSSE(ground_truth_df, forecast_df, train_series_df)
    aggregated_df["rmsse"] = RMSSE(ground_truth_agg_df, forecast_agg_df, train_series_agg_df)

    df_sales["wrmsse"] = df_sales["weight"] * df_sales["rmsse"]
    aggregated_df["wrmsse"] = aggregated_df["weight"] * aggregated_df["rmsse"]

    print(df_sales["wrmsse"].sum() + aggregated_df["wrmsse"].sum())
    
    return (df_sales["wrmsse"].sum() + aggregated_df["wrmsse"].sum())

In [ ]:
%%time

wrmsse=[]
lr=[]
n_leaves=[]
min_d_leaf=[]

for _ in tqdm(range(15)):

    learning_rate = random.randint(25, 75)/1000
    num_leaves = random.randint(32, 128)
    min_data_in_leaf = random.randint(50, 150)

    print(learning_rate)
    print(num_leaves)
    print(min_data_in_leaf)
    
    model = lgb.LGBMRegressor(learning_rate= learning_rate,num_leaves=num_leaves,min_data_in_leaf=min_data_in_leaf)
    
    WRMSSE = hyperparameter_tuning(X_train, y_train, model)

    wrmsse.append(WRMSSE)
    lr.append(learning_rate)
    n_leaves.append(num_leaves)
    min_d_leaf.append(min_data_in_leaf)
   


  0%|          | 0/15 [00:00<?, ?it/s]

0.03
106
139



  7%|▋         | 1/15 [03:54<54:37, 234.11s/it]

0.8616449802156569
0.031
35
70



 13%|█▎        | 2/15 [06:57<47:24, 218.78s/it]

...

0.7983751072002795
0.025
116
117



 93%|█████████▎| 14/15 [48:53<03:38, 218.68s/it]

0.9138323060393725
0.073
64
119



100%|██████████| 15/15 [52:00<00:00, 208.01s/it]

0.7140471507273172
CPU times: user 3h 21s, sys: 1min 46s, total: 3h 2min 7s
Wall time: 52min


In [ ]:
minpos = wrmsse.index(min(wrmsse))
learning_rate = lr[minpos]
num_leaves = n_leaves[minpos]
min_data_in_leaf = min_d_leaf[minpos]

In [27]:
learning_rate = 0.071
num_leaves = 123
min_data_in_leaf = 147

In [28]:
#fit the model on best parameters

m_lgb = lgb.LGBMRegressor( learning_rate=learning_rate ,
                           num_leaves=num_leaves ,
                           min_data_in_leaf=min_data_in_leaf)

m_lgb.fit(X_train, y_train)

LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
              importance_type='split', learning_rate=0.071, max_depth=-1,
              min_child_samples=20, min_child_weight=0.001,
              min_data_in_leaf=147, min_split_gain=0.0, n_estimators=100,
              n_jobs=-1, num_leaves=123, objective=None, random_state=None,
              reg_alpha=0.0, reg_lambda=0.0, silent=True, subsample=1.0,
              subsample_for_bin=200000, subsample_freq=0)

In [29]:
#predict the validation data

for d in range(1914, 1942):
    df_sales['F_' + str(d)] = m_lgb.predict(X_val[X_val['d']==d])

In [30]:
def WRMSSE(df_sales):

  aggregation_level = {2: ["state_id"], 3: ["store_id"], 4: ["cat_id"], 5: ["dept_id"], 
              6: ["state_id", "cat_id"], 7: ["state_id", "dept_id"], 8: ["store_id", "cat_id"], 9: ["store_id", "dept_id"],
              10: ["item_id"], 11: ["item_id", "state_id"]}

  data = df_sales[[a for a in df_sales.columns if a.find("d_") == 0 or a.find("F_") == 0]]
  data = data.sum()

  aggregated_df = pd.DataFrame(data).transpose()    
  aggregated_df["level"] = 1
  aggregated_df["weight"] = 1/12    
  columns = aggregated_df.columns  

  for lev in aggregation_level:
      df_t = df_sales.groupby(by=aggregation_level[lev]).sum().reset_index()
      df_t["level"] = lev
      aggregated_df = aggregated_df.append(df_t[columns])     

  #print(df_sales.shape[0], aggregated_df.shape[0], df_sales.shape[0] + aggregated_df.shape[0])
  #print(aggregated_df["weight"].sum() + df_sales["weight"].sum())    

  train_columns = [a for a in df_sales.columns if a.find("d_") == 0 and int(a.split('_')[1]) < 1914]
  actual_value_columns = [a for a in df_sales.columns if a.find("d_") == 0 and int(a.split('_')[1]) in range(1914, 1942)]
  forecast_value_columns = [a for a in df_sales.columns if a.find("F_") == 0]    


  ground_truth_df = np.array(df_sales[actual_value_columns])
  forecast_df = np.array(df_sales[forecast_value_columns])
  train_series_df = np.array(df_sales[train_columns])

  ground_truth_agg_df = np.array(aggregated_df[actual_value_columns])
  forecast_agg_df = np.array(aggregated_df[forecast_value_columns])
  train_series_agg_df = np.array(aggregated_df[train_columns])

  df_sales["rmsse"] = RMSSE(ground_truth_df, forecast_df, train_series_df)
  aggregated_df["rmsse"] = RMSSE(ground_truth_agg_df, forecast_agg_df, train_series_agg_df)

  df_sales["wrmsse"] = df_sales["weight"] * df_sales["rmsse"]
  aggregated_df["wrmsse"] = aggregated_df["weight"] * aggregated_df["rmsse"]

  print("df", df_sales["wrmsse"].sum())
  print("agg_df",aggregated_df["wrmsse"].sum())

  WRMSSE = df_sales["wrmsse"].sum() + aggregated_df["wrmsse"].sum()
  #print(WRMSSE)

  return WRMSSE     

In [31]:
WRMSSE(df_sales)


df 0.0871697137234316
agg_df 0.619547216432396


0.7067169301558276

**create submission file**

In [32]:
submit_df1 = df_sales[["id"]]

for d, i in enumerate(range(1914, 1942)):
    submit_df1['F' + str(d+1)] = df_sales["F_" + str(i)]

submit_df1["id"] = submit_df1["id"].apply(lambda x: x.replace('evaluation', 'validation'))    

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [34]:
submit_df2 = df_sales[["id"]]
for d in range(1, 29):
    submit_df2['F' + str(d)] = m_lgb.predict(X_test[X_test['d']==(1941+d)])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [35]:
submit = submit_df1.append(submit_df2).reset_index(drop=True)
submit.to_csv("submission.csv", index=False)